<center><h1 style="color:#D4AF37"> ⚡⚡ AEMR Data Analysis ⚡⚡</h1>

# <span style="color: green;">What's the Business Problem? 💰</span> 
The American Energy Market Regulator (AEMR) is responsible for looking after the United States of America’s domestic energy network. The regulator’s responsibility is to ensure that America’s energy network remains reliable with minimal disruptions, which are known as outages.

There are four key types of outages:

● `Consequential` – This outage is caused by an exogenous event or a
consequence of work unrelated to the energy provider (i.e. A transmission line
may be down, which means that a particular power provider may be unable to
send energy to the network)

● `Forced` – This outage is caused by a situation that has forced the power
generating asset to be unavailable. In other words, it has caught the AEMR by
surprise and was not planned. (i.e. A cooling tower that supplies water to cool
down a power-generating asset has broken down, which has now forced the
power-generating asset offline as the temperature can no longer be regulated
properly.)

● `Opportunistic` – This outage arises when an energy provider wishes to be
proactive with the maintenance of their assets and believes that it can complete
maintenance on its plant within a 48-hour window. If it takes longer than this, the
outage is considered a PLANNED outage as it was planned in advance and will
take longer than 48 hours.

● `Planned` – This outage arises when an energy provider reports to the AEMR that
an energy-generating asset needs to be taken offline for routine or planned
maintenance activities to ensure the reliability of the asset in the future.
Of the four outage types, the only one the AEMR penalizes is a forced outage, as this
means that, if enough energy providers enter the forced outage state and the demand
for energy is greater than the amount of energy that can be supplied, stress will be put
on the energy system. This will then threaten the reliability of the network which is what
the AEMR wishes to avoid.

Of the four outage types, the only one the AEMR penalizes is a forced outage, as this
means that, if enough energy providers enter the forced outage state and the demand
for energy is greater than the amount of energy that can be supplied, stress will be put
on the energy system. This will then threaten the reliability of the network which is what
the AEMR wishes to avoid.

Recently, the AEMR management team has been increasingly aware of a large number of energy providers that submitted outages over 
the 2016 and 2017 calendar years. The management team has expressed a desire to have the following two areas of concern addressed:

A) Energy Stability and Market Outages

B) Energy Losses and Market Reliability

As an analyst within the data and reporting team, we've been asked to address these two immediate areas of concern

In [ ]:
%%capture
!pip install ipython-sql sqlalchemy
import sqlalchemy
sqlalchemy.create_engine("sqlite:///AEMR.db")
%load_ext sql
%sql sqlite:///AEMR.db

In [ ]:
%%js
require(['notebook/js/codecell'], function (codecell) {
    codecell.CodeCell.options_default.highlight_modes['magic_text/x-mssql'] = { 'reg': [/%?%sql/] };
    Jupyter.notebook.events.one('kernel_ready.Kernel', function () {
        Jupyter.notebook.get_cells().map(function (cell) {
            if (cell.cell_type == 'code') { cell.auto_highlight(); }
        });
    });
});

<IPython.core.display.Javascript object>

### <span style="color: green;">Energy stability is one of the key themes the AEMR management team cares about To ensure energy security and reliability, AEMR needs to understand the following:</span> 

* **What are the most common outage types and how long do they tend to last?**
* **How frequently do the outages occur?**
* **Are there any energy providers which have more outages than their peers which may be indicative of being unreliable?**
### <span style="color: green;">Please note that throughout the entire analysis, we are interested ONLY in the Outages where Status = Approved.</span> 
### <span style="color: green;">We don't have any interest in Outages that were cancelled or not approved.</span>
### <span style="color: green;">This means our WHERE Clause will ALWAYS contain the field Where Status = Approved.</span> 

<h3 style="color:#D4AF37"> Step One </h3>

COUNT the number of valid (i.e. Status = Approved) Outage Events sorted by their respective Outage_Reason i.e. `Forced`, `Consequential`, `Scheduled`, `Opportunistic`) over the 2016 & 2017 Periods.


In [ ]:
%%sql
SELECT COUNT(*) as Total_Number_Outages,
 Outage_Reason, year
FROM AEMR_Outage_Table
WHERE status = 'Approved'
GROUP BY Outage_Reason,year
LIMIT 10

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Outage_Reason,Year
181,Consequential,2016
127,Consequential,2017
1264,Forced,2016
1622,Forced,2017
106,Opportunistic Maintenance (Planned),2016
102,Opportunistic Maintenance (Planned),2017
380,Scheduled (Planned),2016
320,Scheduled (Planned),2017


<h3 style="color:#D4AF37"> Step Two </h3>



i) Show the Total of all Outage Types (`Forced`, `Consequential`, `Scheduled`, `Opportunistic`) where the Status = Approved, that occurred for both 2016 and 2017, grouped by Year and Month. per month (i.e. 1 – 12). Order by Year, Month, Total_Number_Outages in Descending Order.

ii) Building on the query we wrote in i), group the results by Outage Type, Year and Month. This is so we can identify whether there is any outage type specifically increasing on a monthly basis when comparing 2016 to 2017. 

In [ ]:
%%sql
SELECT
COUNT (*) AS Total_Number_Outages,
year,
month
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY year, month
LIMIT 10


 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Year,Month
191,2016,1
227,2016,2
136,2016,3
134,2016,4
174,2016,5
168,2016,6
147,2016,7
194,2016,8
124,2016,9
124,2016,10


<h3 style="color:#D4AF37"> Step Three </h3>


Calculate 1) The `Total_Number_Outage_Events` and 2) The `Average Duration in DAYS` for each Participant Code and Outage Type over the 2016 and 2017 Period where the Status = Approved. Order by `Total_Number_Outage_Events` in Descending Order, Reason and Year.


In [ ]:
%%sql
SELECT
COUNT (*) AS Total_Number_Outage_Events,
ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) AS Average_Outage_Duration_In_Days,
Participant_Code,
Outage_Reason,
year
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Participant_Code, Outage_Reason, year
ORDER BY Total_Number_Outage_Events DESC
LIMIT 10



 * sqlite:///AEMR.db
Done.


Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Participant_Code,Outage_Reason,Year
490,0.07,AURICON,Forced,2017
317,0.38,GW,Forced,2016
227,1.06,GW,Forced,2017
208,0.07,AURICON,Forced,2016
206,0.08,AUXC,Forced,2016
177,2.28,MELK,Forced,2017
172,0.42,TRMOS,Forced,2017
157,0.83,MELK,Forced,2016
135,0.25,PUG,Forced,2017
120,0.02,AUXC,Forced,2017


Now we're getting somewhere...!
We've identified participants who are having many outages, as well as participants who have been offline for the longest durations.

Armed with this information, it's important we're able to classify our participants accordingly based on reliability metrics of uptime.

We classify a participant based off the following criteria:
<b>
**<li> High Risk - On average, the participant is unavailable for > 24 Hours (1 Day)</li>**
**<li> Medium Risk - On average, the participant is unavailable between 12 and 24 Hours </li>**
**<li> Low Risk - On average, the participant is unavailable for less than 12 Hours</li>**
</b>

<h3 style="color:#D4AF37"> Step Four </h3>

Classify each participant code as either `High Risk`, `Medium Risk` or `Low Risk` in a column called `Risk_Classification` that is based off their Average Outage Duration Time. Please note that this is for all valid (i.e. Where status = approved) outage types (`Forced`, `Consequential`, `Scheduled`, `Opportunistic`) for all participant codes from 2016 to 2017. Order the results using `Average Duration Time In Days` in descending order.
                                                                                                                                                                                                                         

In [ ]:
%%sql
WITH CTE1 AS (
SELECT
COUNT (*) AS Total_Number_Outage_Events,
ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) AS Average_Outage_Duration_In_Days,
Participant_Code,
Outage_Reason,
year
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Participant_Code, Outage_Reason, year
ORDER BY Total_Number_Outage_Events DESC

)

SELECT *,
CASE
    WHEN Average_Outage_Duration_In_Days > 1 THEN "High Risk"
    WHEN Average_Outage_Duration_In_Days  BETWEEN .5 AND 1 THEN "Medium Risk"
    WHEN Average_Outage_Duration_In_Days  BETWEEN 0 AND .5 THEN "Low Risk"
    ELSE 'N/A'
    END AS Risk_Classification

FROM CTE1;

 * sqlite:///AEMR.db
Done.


Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Participant_Code,Outage_Reason,year,Risk_Classification
490,0.07,AURICON,Forced,2017,Low Risk
317,0.38,GW,Forced,2016,Low Risk
227,1.06,GW,Forced,2017,High Risk
208,0.07,AURICON,Forced,2016,Low Risk
206,0.08,AUXC,Forced,2016,Low Risk
177,2.28,MELK,Forced,2017,High Risk
172,0.42,TRMOS,Forced,2017,Low Risk
157,0.83,MELK,Forced,2016,Medium Risk
135,0.25,PUG,Forced,2017,Low Risk
120,0.02,AUXC,Forced,2017,Low Risk


Now that we've classified our participants as either `High Risk`, `Medium Risk` or `Low Risk`, we want to dig a little deeper.

Does it make sense that `Consequential`, `Opportunistic` or `Planned` aren't considered regarding the Risk Category?

Perhaps we should refine our category accordingly by ensuring we focus our Risk Category on labelling only `Forced` Outages as being a Risk. After all, Forced Outages are the unplanned outages that risk the security of the electricity grid.

Let's add two additional criteria to our classification considering `Total Number of Outage Events` and `Outage Type`.

We've summarised these below: 

<b>
<li> High Risk - On average, the participant is unavailable for > 24 Hours (1 Day) OR the Total Number of Outage Events > 20 </li>
<li> Medium Risk - On average, the participant is unavailable between 12 and 24 Hours OR the Total Number of Outage Events is Between 10 and 20 </li>
<li> Low Risk - On average, the participant is unavailable for less than 12 Hours OR the Total Number of Outage Events < 10 </li>
<li> If Outage Type is not forced, then N/A
</b>


Classifies each participant code as either High Risk, Medium Risk or Low Risk in a column called Risk_Classification using the new classification criteria. Order the results using Average Duration Time In Days in descending order.

In [ ]:
%%sql
WITH CTE1 AS (
SELECT
COUNT (*) AS Total_Number_Outage_Events,
ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) AS Average_Outage_Duration_In_Days,
Participant_Code,
Outage_Reason,
year
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Participant_Code, Outage_Reason, year
ORDER BY Total_Number_Outage_Events DESC

)

SELECT *,
CASE
    WHEN Average_Outage_Duration_In_Days > 1 AND Outage_Reason = 'Forced' OR Total_Number_Outage_Events > 20 AND Outage_Reason = 'Forced' THEN "High Risk"
    WHEN Average_Outage_Duration_In_Days  BETWEEN .5 AND 1 and Outage_Reason = 'Forced' OR Total_Number_Outage_Events BETWEEN 10 AND 20 AND Outage_Reason = 'Forced' THEN "Medium Risk"
    WHEN Average_Outage_Duration_In_Days  BETWEEN 0 AND .5 THEN "Low Risk"
    ELSE 'N/A'
    END AS Risk_Classification

FROM CTE1;

 * sqlite:///AEMR.db
Done.


Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Participant_Code,Outage_Reason,year,Risk_Classification
490,0.07,AURICON,Forced,2017,High Risk
317,0.38,GW,Forced,2016,High Risk
227,1.06,GW,Forced,2017,High Risk
208,0.07,AURICON,Forced,2016,High Risk
206,0.08,AUXC,Forced,2016,High Risk
177,2.28,MELK,Forced,2017,High Risk
172,0.42,TRMOS,Forced,2017,High Risk
157,0.83,MELK,Forced,2016,High Risk
135,0.25,PUG,Forced,2017,High Risk
120,0.02,AUXC,Forced,2017,High Risk


<h3 style="color:#D4AF37"> ⚡ Part II. Energy Losses & Market Reliability ⚡ </h3>

When an energy provider provides energy to the market, they are making a commitment to the market and saying; “We will supply X amount of energy to the market under a contractual obligation.” However, in a situation where the outages are Forced, the energy provider intended to provide energy but are unable to provide energy and are forced offline. If many energy providers are forced offline at the same time it could cause an energy security risk that AEMR needs to mitigate.

To ensure this doesn’t happen, the AEMR is interested in exploring the following questions:

Of the outage types in 2016 and 2017, what percent were Forced Outage(s)?
What was the average duration for a forced outage during both 2016 and 2017? Have we seen an increase in the average duration of forced outages?
Which energy providers tended to have the largest number of forced outages?

<h3 style="color:#D4AF37"> Step Six </h3>

Calculate the proportion of Forced Outages that have occurred over the 2016 - 2017 Period.
Do we observe any particular increases regarding any Outage Types over this period? 

In [ ]:
%%sql

SELECT
COUNT(*) AS Total_Number_Outages,
SUM(CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE 0 END)  AS  Total_Number_Forced_Outage_Events,
ROUND(SUM(CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE 0 END)  * 1.0/COUNT(*)*100,2) AS Pct_Forced_Outages,
Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY year



 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Total_Number_Forced_Outage_Events,Pct_Forced_Outages,Year
1931,1264,65.46,2016
2171,1622,74.71,2017


Great. It's clear to see now that `Forced Outages` are problematic for us. Not only are they the only outage type that generates financial losses as the Outage is unplanned, it seems there is a number of Energy Participants who have been having a significantly high number of Outages.

Let's break our analysis down into Macro and Micro Analysis.
The total gives us the Overall Duration a participant is offline / has lost energy, however, it doesn't tell us how *frequently* this occurs. In other words, if we have one or two very big outages, it might contribute to very large totals.

<h3 style="color:#D4AF37"> Step Seven </h3>

Calculate the `Total Number of Outages`, `Total Duration In Days`and `Total Energy Lost` of all valid `Outages` for each `participant code` and `facility_code`, sorted by `Total Energy Lost` in descending order and Ordered by the YEAR Category.
    

In [ ]:
%%sql

SELECT
COUNT(*) AS Total_Number_Outages,
SUM(ABS((JULIANDAY(End_Time) - JULIANDAY(Start_Time)))) AS Total_Duration_In_Days,
SUM(Energy_Lost_MW) AS Total_Energy_Lost,
Outage_Reason,
Participant_Code,
Facility_Code,
Year

FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Outage_Reason, Participant_code, Facility_Code, Year
ORDER BY Total_Duration_In_Days, Year




 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Total_Duration_In_Days,Total_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
1,0.04166666651144624,36.0,Consequential,WGUTD,WEST_KALGOORLIE_GT2,2017
2,0.041666666977107525,54.0,Forced,WGUTD,WEST_KALGOORLIE_GT2,2016
1,0.0625,0.0,Opportunistic Maintenance (Planned),TRMOS,TIWEST_COG1,2016
1,0.0625,23.0,Opportunistic Maintenance (Planned),STHRNCRS,STHRNCRS_EG,2017
1,0.08333333348855376,335.0,Opportunistic Maintenance (Planned),PMC,PMC_AG,2016
3,0.10416666651144624,64.80000000000001,Forced,EUCT,GRASMERE_WF1,2017
2,0.10416666651144624,72.0,Opportunistic Maintenance (Planned),WGUTD,WEST_KALGOORLIE_GT2,2017
1,0.10416666697710752,26.0,Consequential,AUXC,AUXC_WGP,2017
4,0.22916666651144624,220.0,Scheduled (Planned),MCG,MWF_MUMBIDA_WF1,2017
1,0.25,335.0,Opportunistic Maintenance (Planned),PMC,PMC_AG,2017


<h3 style="color:#D4AF37"> Step Eight </h3>

Calculate the `Average Duration In Days` and Average Energy Lost of all valid `FORCED OUTAGES` for each `participant_code` and `facility_code` sorted by Average Energy Lost in descending order and Ordered by the YEAR Category.
    

In [ ]:
%%sql

SELECT
ROUND(AVG(ABS((JULIANDAY(End_Time) - JULIANDAY(Start_Time)))),2) AS Avg_Duration_In_Days,
ROUND(AVG(Energy_lost_MW),2) AS Avg_Energy_Lost,
Outage_Reason,
Participant_Code,
Facility_code,
Year

FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Outage_reason, Participant_code, Facility_Code, Year
ORDER BY Avg_Energy_Lost, Year DESC
LIMIT 5


 * sqlite:///AEMR.db
Done.


Avg_Duration_In_Days,Avg_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
0.06,0.0,Opportunistic Maintenance (Planned),TRMOS,TIWEST_COG1,2016
0.24,1.44,Consequential,DNHR,DNHR_DENMARK_WF1,2017
0.48,1.44,Forced,DNHR,DNHR_DENMARK_WF1,2017
0.28,1.44,Consequential,DNHR,DNHR_DENMARK_WF1,2016
0.4,1.44,Forced,DNHR,DNHR_DENMARK_WF1,2016


<h3 style="color:#D4AF37"> Step Nine </h3>

Calculate the `Average Energy Lost` and `Total Energy Lost` for each `Facility Code` and `Participant Code` across both the 2016 and 2017 periods when the Outage_Reason is set to Forced. Upon completion of this, calculate the percentage of energy lost due to forced outages for each Facility_Code. Please ORDER the query by Total Energy Lost from 2016 to 2017.
                                                                                                                                                                                                                                   
Which participants have contributed the most to the Energy Lost due to Forced Outages? 
                                                                                                                                                                                                          

In [ ]:
%%sql

SELECT
ROUND(AVG(Energy_Lost_MW),2) AS Avg_Energy_Lost,
ROUND(SUM(Energy_Lost_MW),2) AS Total_Energy_Lost,
ROUND(SUM(Energy_Lost_MW) / (SELECT SUM(ENERGY_LOST_MW) FROM AEMR_Outage_Table WHERE Status = 'Approved' AND Outage_Reason = 'Forced')*100,2) AS Pct_Energy_Lost,
Outage_Reason,
Participant_Code,
Facility_Code,
Year

FROM AEMR_Outage_Table
WHERE Status = 'Approved' AND Outage_Reason = 'Forced'
GROUP BY Outage_Reason, Participant_Code, Facility_Code, Year
ORDER BY Total_Energy_Lost DESC, Year DESC
LIMIT 5

 * sqlite:///AEMR.db
Done.


Avg_Energy_Lost,Total_Energy_Lost,Pct_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
44.16,21639.55,14.21,Forced,AURICON,AURICON_PNJ_U1,2017
85.14,19326.56,12.69,Forced,GW,BW1_GREENWATERS_G2,2017
49.69,15751.38,10.34,Forced,GW,BW1_GREENWATERS_G2,2016
87.71,13771.07,9.04,Forced,MELK,MELK_G7,2016
51.42,10696.28,7.02,Forced,AURICON,AURICON_PNJ_U1,2016


<h3 style="color:#D4AF37"> Step Ten </h3>

Having identified the top 3 participants by Total Energy Loss being `GW`, `MELK` and `Auricon`; we now need to calculate the `Total_Energy_Lost` each of these three `Participant Codes` and the `Facility_Code`. Additionally, identify the Description_Of_Outage associated with the highest Total_Energy_Lost for each of the Participant_Codes and Facility_Code for each of the three participants.

Lastly, calculate the percentage of Energy Loss, attributed to these reasons!

In [ ]:
%%sql
SELECT
*
FROM (
SELECT
Participant_Code,
Facility_Code,
Description_Of_Outage,
ROUND(SUM(Energy_Lost_MW),2) AS Total_Energy_Lost,
ROUND(SUM(Energy_Lost_MW) / (SELECT SUM(Energy_Lost_MW) FROM AEMR_Outage_Table WHERE Status = 'Approved' AND Outage_Reason = 'Forced')*100,2)As Pct_Forced,
RANK() OVER (PARTITION BY Participant_Code, Facility_code ORDER BY SUM(Energy_Lost_MW) DESC) AS Ranking
FROM AEMR_Outage_Table
WHERE Participant_Code IN ('GW', 'MELK', 'AURICON') AND Status = 'Approved' AND Outage_Reason = 'Forced'
GROUP BY Participant_Code, Facility_Code, Description_Of_Outage
ORDER BY Participant_Code, Total_Energy_Lost DESC)

WHERE Ranking = 1



 * sqlite:///AEMR.db
Done.


Participant_Code,Facility_Code,Description_Of_Outage,Total_Energy_Lost,Pct_Forced,Ranking
AURICON,AURICON_PNJ_U1,Full unit trip,6033.87,3.96,1
GW,BW1_GREENWATERS_G2,Operational Issues caused real time forced outage.,28687.54,18.84,1
MELK,MELK_G7,Safety Issues,1100.0,0.72,1
